In [1]:
import pandas as pd
import numpy as np

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver

from bs4 import BeautifulSoup 
import requests 

from tqdm import tnrange, tqdm_notebook as tqdm
import time
import random
from copy import deepcopy

import pickle

In [5]:
df = pd.read_pickle('bgg_full_df_600.pkl')

In [41]:
df.head()

,0,1,2,name,overall_rank,category_rank,no_of_players,playing_time,complexity,prices,categories,mechanism,category_rank_cat,no_of_players_suggested,no_of_players_best_rec
0,7.1,Thurn and Taxis (2006),17K Ratings & 3.9K Comments · GeekBuddy Analysis,7.1\nThurn and Taxis (2006)\n17K Ratings & 3.9...,402,281,2–4 Players\nCommunity: 2–4 — Best: 3–4,60 Min\nPlaying Time,Weight: 2.26 / 5\nComplexity Rating\n'Complexi...,STORES\n(unavailable)\nAmazon.com\nEbay listings,Categories\nPost-Napoleonic\nTransportation,Mechanisms\nCard Drafting\nHand Management\nNe...,STRATEGY,2–4 Players,3–4
1,7.5,Cosmic Encounter (2008),26K Ratings & 5.4K Comments · GeekBuddy Analysis,7.5\nCosmic Encounter (2008)\n26K Ratings & 5....,130,50,3–5 Players\nCommunity: 4–5+ — Best: 5,60–120 Min\nPlaying Time,Weight: 2.57 / 5\nComplexity Rating\n'Complexi...,STORES\n$59.95\nSuggested retail\n$53.95\nAmaz...,Categories\nBluffing\nNegotiation\nScience Fic...,Mechanisms\nAlliances\nAuction/Bidding\nHand M...,THEMATIC,3–5 Players,5
2,7.1,Eminent Domain (2011),9K Ratings & 2.3K Comments · GeekBuddy Analysis,7.1\nEminent Domain (2011)\n9K Ratings & 2.3K ...,515,334,2–4 Players\nCommunity: 2–4 — Best: 3,45 Min\nPlaying Time,Weight: 2.50 / 5\nComplexity Rating\n'Complexi...,STORES\n$39.99\nSuggested retail\n$29.82\nAmaz...,Categories\nCard Game\nCivilization\nScience F...,"Mechanisms\nCard Drafting\nDeck, Bag, and Pool...",STRATEGY,2–4 Players,3
3,7.9,Yokohama (2016),7.4K Ratings & 1.4K Comments · GeekBuddy Analysis,7.9\nYokohama (2016)\n7.4K Ratings & 1.4K Comm...,93,65,2–4 Players\nCommunity: 2–4 — Best: 3,90 Min\nPlaying Time,Weight: 3.31 / 5\nComplexity Rating\n'Complexi...,STORES\n$59.95\nSuggested retail\n$51.61\nAmaz...,Category\nEconomic,Mechanisms\nContracts\nEnd Game Bonuses\nGrid ...,STRATEGY,2–4 Players,3
4,8.3,Great Western Trail (2016),24K Ratings & 3.3K Comments · GeekBuddy Analysis,8.3\nGreat Western Trail (2016)\n24K Ratings &...,10,10,2–4 Players\nCommunity: 2–4 — Best: 3,75–150 Min\nPlaying Time,Weight: 3.70 / 5\nComplexity Rating\n'Complexi...,STORES\n(unavailable)\nAmazon.com\nEbay listings,Categories\nAmerican West\nAnimals,"Mechanisms\nDeck, Bag, and Pool Building\nHand...",STRATEGY,2–4 Players,3


In [9]:
names = df['name'].str.split('\n', expand =True)

In [11]:
df = names.merge(df, how='inner', left_index=True, right_index=True)

In [19]:
df['overall_rank'] = df['overall_rank'].str.split(' ', expand =True)[1]

In [27]:
df['category_rank_cat'] = df['category_rank'].str.split(' ', expand= True)[0]

In [28]:
df['category_rank'] = df['category_rank'].str.split(' ', expand= True)[1]

In [34]:
df['no_of_players_suggested'] = df['no_of_players'].str.split('\n', expand =True)[0]

In [40]:
df['no_of_players_best_rec']= df['no_of_players'].str.split('\n', expand =True)[1].str.split('Best: ', expand =True)[1]

In [47]:
df['playing_time'] = df['playing_time'].str.split('\n', expand = True)[0]

In [52]:
df['complexity'] = df['complexity'].str.split('\n', expand =True)[0]

In [55]:
df['categories'].str.split('\n')

0          [Categories, Post-Napoleonic, Transportation]
1      [Categories, Bluffing, Negotiation, Science Fi...
2      [Categories, Card Game, Civilization, Science ...
3                                   [Category, Economic]
4                   [Categories, American West, Animals]
                             ...                        
594                    [Categories, Card Game, Medieval]
595    [Categories, Card Game, Civilization, Economic...
596                  [Categories, Wargame, World War II]
597    [Categories, Economic, Pike and Shot, Renaissa...
598           [Categories, Card Game, Deduction, Memory]
Name: categories, Length: 599, dtype: object